<a href="https://colab.research.google.com/github/zappy586/Phi-2-MATH/blob/main/Phi2_MATH_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading dependencies

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U xformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U einops
!pip install -q -U nvidia-ml-py3
!pip install -q -U huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━

# Loading the dataset
### The Dataset we are using is orca-math-word-problems-200k which has 200k QA pairs of mathematical word problems. We will be finetuning the Phi-2 LLM on this

In [ ]:
from datasets import load_dataset
#
dataset = load_dataset("microsoft/orca-math-word-problems-200k")
dataset

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 200035
    })
})

In [ ]:
def create_prompt(sample):
  system_prompt_template = """<s>
Below is an instruction that describes a math problem.Write a response that appropriately and accurately solves the math problem.
### Instruction :<<user_question>>
### Response:
<<user_response>>
</s>
"""
  user_message = sample['question']
  user_response = sample['answer']
  prompt_template = system_prompt_template.replace("<<user_question>>",f"{user_message}").replace("<<user_response>>",f"{user_response}")

  return {"inputs":prompt_template}

#
instruct_tune_dataset = dataset.map(create_prompt)
print(instruct_tune_dataset)

Map:   0%|          | 0/200035 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'inputs'],
        num_rows: 200035
    })
})


In [ ]:
instruct_tune_dataset['train'][0]

{'question': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.',
 'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.',
 'inputs': '<s>\nBelow is an instruction that describes a math problem.Write a response that appropriately and accurately solves the math problem.\n### Instruction :Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.\n### Response:\nIf Jungkook is in 5th place, then 4 people crossed the finish line faster than him.\n</s>\n'}

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from pynvml import *
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import time, torch

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
# base_model_id = "microsoft/phi-2"

# #Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_model_id  , use_fast=True)
# #Load the model with fp16
# model =  AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map={"": 0})
# print(print_gpu_utilization())

In [ ]:
# duration = 0.0
# total_length = 0
# prompt = []
# prompt.append("Write the recipe for a chicken curry with coconut milk.")
# prompt.append("Translate into French the following sentence: I love bread and cheese!")
# prompt.append("Cite 20 famous people.")
# prompt.append("Where is the moon right now?")

# for i in range(len(prompt)):
#   model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
#   start_time = time.time()
#   # with torch.autocast(model.device.type, dtype=torch.float16, enabled=True):
#   output = model.generate(**model_inputs, max_length=500)[0]
#   duration += float(time.time() - start_time)
#   total_length += len(output)
#   tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
#   print("Prompt --- %s tokens/seconds ---" % (tok_sec_prompt))
#   print(print_gpu_utilization())
#   print(tokenizer.decode(output, skip_special_tokens=True))

# tok_sec = round(total_length/duration,3)
# print("Average --- %s tokens/seconds ---" % (tok_sec))

# Loading the base model checkpoint


In [ ]:
base_model_id = "microsoft/phi-2"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True, use_fast=True, max_length=250)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

compute_dtype = getattr(torch, "bfloat16") #change to bfloat16 if are using an Ampere (or more recent) GPU
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          base_model_id, trust_remote_code=True, quantization_config=bnb_config, revision="refs/pr/23", device_map={"": 0}, torch_dtype="auto", flash_attn=True, flash_rotary=True, fused_dense=True
)
print(print_gpu_utilization())

model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


GPU memory occupied: 2723 MB.
None


# Creating the PEFT config

### PEFT refers to parameter efficient fine-tuning which is a finetuning technique for LLMs which uses significantly less compute

### This technique uses something called LoRA(Low rank adaptation) which decomposes the changes made to the network's weight matrix into a product of two lower-rank matrices, A and B. This decomposition reduces computational overhead while preserving the model's learning capability.

In [ ]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["out_proj", "Wqkv"])

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./phi2-results2",
        save_strategy="epoch",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=12,
        log_level="debug",
        save_steps=100,
        logging_steps=25,
        learning_rate=1e-4,
        eval_steps=50,
        optim='paged_adamw_8bit',
        fp16=True, #change to bf16 if are using an Ampere GPU
        num_train_epochs=5,
        # max_steps=500,
        warmup_steps=100,
        lr_scheduler_type="linear",
        seed=42)

In [ ]:
train_dataset = instruct_tune_dataset.map(batched=True,remove_columns=['answer', 'question'])
train_dataset

Map:   0%|          | 0/200035 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['inputs'],
        num_rows: 200035
    })
})

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset['train'],
        #eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="inputs",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
        packing=False
)


Map:   0%|          | 0/200035 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Using auto half precision backend


In [ ]:
trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 200,035
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 12
  Total optimization steps = 20,835
  Number of trainable parameters = 7,864,320


Step,Training Loss


KeyboardInterrupt: 

# Pushing the finetuned adapters to HF Hub

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from huggingface_hub import notebook_login

notebook_login()

trainer.push_to_hub(commit_message="fine-tuned adapter")

Saving model checkpoint to ./phi2-results2
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/config.json
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
Model config PhiConfig {
  "_name_or_path": "microsoft/phi-2",
  "activation_function": "gelu_new",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attn_pdrop": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/phi-2--configuration_phi.PhiConfig",
    "AutoModelForCausalLM": "microsoft/phi-2--modeling_phi.PhiForCausalLM"
  },
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "flash_attn": false,
  "flash_rotary": false,
  "fused_dense": false,
  "hidden_act": "gelu_new",
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "layer_norm_epsilo

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

events.out.tfevents.1712211287.02ad155b663d.751.0:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1712211541.02ad155b663d.751.1:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ZappY-AI/phi2-results2/commit/8a445f5e703fe85b4ebc48e9685700dd0cddeed9', commit_message='fine-tuned adapter', commit_description='', oid='8a445f5e703fe85b4ebc48e9685700dd0cddeed9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# import gc
# del model
# del tokenizer
# del trainer
# del train_dataset
# del peft_config
# del training_arguments
# gc.collect()

In [ ]:
# print(torch.cuda.memory_summary(device=None, abbreviated=False))

# Inference using the newly fine-tuned adapters

In [ ]:
base_model_id = "microsoft/phi-2"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          base_model_id, trust_remote_code=True, quantization_config=bnb_config, device_map={"": 0}
)
adapter = "/content/drive/MyDrive/PHI2/checkpoint-500"
model = PeftModel.from_pretrained(model, adapter)

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/tokenizer_config.json
Specia

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing PhiForCausalLM.

All the weights of PhiForCausalLM were initialized from the model checkpoint at microsoft/phi-2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use PhiForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}



In [ ]:
question = "Sam has 30 nickels, dimes, and quarters worth 5.70 dollars. How many nickels does he have?"
prompt_template1 = f""""
Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :
{question}
### Response:
"""
prompt_template1

'"\nBelow is an instruction that describes a task.Write a response that appropriately completes the request.\n### Instruction :\nSam has 30 nickels, dimes, and quarters worth 5.70 dollars. How many nickels does he have?\n### Response:\n'

In [ ]:
duration = 0.0
total_length = 0
prompt = []
prompt.append(prompt_template1)
#
for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500, no_repeat_ngram_size=10, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/seconds ---" % (tok_sec_prompt))
  print(print_gpu_utilization())
  print(tokenizer.decode(output, skip_special_tokens=False))

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/seconds ---" % (tok_sec))

Prompt --- 9.827 tokens/seconds ---
GPU memory occupied: 7744 MB.
None
"
Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :
Sam has 30 nickels, dimes, and quarters worth 5.70 dollars. How many nickels does he have?
### Response:
Sam has 15 nickels.

<|question_end|>Solution:

Question 1:
a) The number of students in the class is 30.
b) The number of students who like to play football is 15.
c) The number of students who like to play basketball is 10.
d) The number of students who like to play cricket is 5.
e) The number of students who like to play tennis is 5.

Question 2:
a) The number of students in class A is 30.
b) The number of boys in class A is 15.
c) The number of girls in class A is 15.
d) The number of students in class B is 30.
e) The number of boys in class B is 15.
f) The number of girls in class B is 15.

Question 3:
a) The number of students in Sam's class is 30.
b) The number and value of nickels S

# Merging the adapters with the base model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#
import shutil
shutil.move('/content/phi2-results2', '/content/drive/MyDrive/PHI2')

Mounted at /content/drive


'/content/drive/MyDrive/PHI2'

In [ ]:
from peft import AutoPeftModelForCausalLM
trained_model = AutoPeftModelForCausalLM.from_pretrained("/content/drive/MyDrive/PHI2/checkpoint-500",
                                                         low_cpu_mem_usage=True,
                                                         return_dict=True,
                                                         torch_dtype=torch.float16,
                                                         device_map='auto',)
#
lora_merged_model = trained_model.merge_and_unload()
#
# Save the merged Model into drive
lora_merged_model.save_pretrained("/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model",safe_serialization=True)
# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/config.json
Model config PhiConfig {
  "_name_or_path": "microsoft/phi-2",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/phi-2--configuration_phi.PhiConfig",
    "AutoModelForCausalLM": "microsoft/phi-2--modeling_phi.PhiForCausalLM"
  },
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "hidden_act": "gelu_new",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "phi",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "partial_rotary_factor": 0.4,
  "qk_layernorm": false,
  "resid_pdrop": 0.1,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "f

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing PhiForCausalLM.

All the weights of PhiForCausalLM were initialized from the model checkpoint at microsoft/phi-2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use PhiForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This me

('/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model/tokenizer_config.json',
 '/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model/special_tokens_map.json',
 '/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model/vocab.json',
 '/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model/merges.txt',
 '/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model/added_tokens.json',
 '/content/drive/MyDrive/PHI2/phi2-results2/lora_merged_model/tokenizer.json')

# Pushing the merged model to a HF repo

In [ ]:
lora_merged_model.push_to_hub(repo_id="ZappY-AI/phi2-results2",commit_message="merged model")

README.md:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Configuration saved in /tmp/tmpc1_sg3_e/config.json
Configuration saved in /tmp/tmpc1_sg3_e/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at /tmp/tmpc1_sg3_e/model.safetensors.index.json.
Uploading the following files to ZappY-AI/phi2-results2: config.json,model-00002-of-00002.safetensors,README.md,model-00001-of-00002.safetensors,generation_config.json,model.safetensors.index.json


model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ZappY-AI/phi2-results2/commit/241245716d9d766be4d6117425f5da4c3fa20f60', commit_message='merged model', commit_description='', oid='241245716d9d766be4d6117425f5da4c3fa20f60', pr_url=None, pr_revision=None, pr_num=None)